# Imports

In [ ]:
import os, sys
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))

import hpc05
import holoviews as hv

import numpy as np
import kwant
import scipy.constants
import cmath
import functools as ft
from itertools import product

import plotting_results
import adaptive
adaptive.notebook_extension()

import sns_system, spectrum, topology, supercurrent

from IPython.core.magic import register_cell_magic
import ipywidgets as widgets
run_cluster = widgets.Checkbox(
    value=False,
    description='Check to run cluster',
    disabled=False
)

interactive_shell = get_ipython()
@register_cell_magic
def cluster_mark_checked(line, cell):
    if run_cluster.value is True:
        run_cluster.value = False
        interactive_shell.run_cell(cell)
    else:
        return "Command not run. Check the box above to run."

del(cluster_mark_checked)
run_cluster

In [ ]:
%%cluster_mark_checked
# client, dview, lview = hpc05.connect_ipcluster(20, folder='two_dim_majoranas')
# hpc05.kill_remote_ipcluster()
client, dview, lview = hpc05.start_remote_and_connect(80, folder='two_dim_majoranas')
client[:].use_pickle() 

In [ ]:
%%px
import numpy as np
import scipy.constants
import cmath

import functools as ft
import kwant
import sns_system, spectrum, topology, supercurrent


In [ ]:
%%px
constants = dict(
    m_eff=0.02 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

# Make zigzag shaped system

In [ ]:
%%px
params_raw= dict(g_factor_middle = 26,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 10,
                 alpha_middle = 20,
                 alpha_left = 0,
                 alpha_right = 0,
                 Delta_left = 2,
                 Delta_right = 2,
                 B = 0.4,
                 phase = np.pi,
                 T = 0.0,
                 V = 0)

syst_pars = dict(a=5, L_m=500, L_x=4000, W_up=200, W_down=200, z_x=800, z_y=300,
                 offset_lead=100, k_x_in_sc=True, leaded=False, edge_thickness=0)


params = dict(**constants,
              **params_raw)

syst=sns_system.make_zigzag_system(**syst_pars)
# plotting_results.plot_syst(syst=syst, params=dict(**params));
# plt.savefig('syst_with_edge', dpi=2000)

In [ ]:
syst_pars_ns = dict(a=5, L_m=15, L_x=20, L_up=200, L_down=200,
                  k_x_in_sc=True, )
syst_ns = sns_system.make_ns_junction(**syst_pars_ns)
plotting_results.plot_syst(syst=syst_ns, params=dict(**params, k_x=0));
scattering.transparency(syst_ns, params)
# 0.71397506386568876

In [ ]:
ham = syst.hamiltonian_submatrix(params=dict(**params), sparse=True)
d = kwant.operator.Density(syst)
e, ev = spectrum.sparse_diag(ham,4,0)

In [ ]:
e

In [ ]:
%%px
def vary_mu_B(muB):
    mu, B = muB
    _params = params.copy()
    _params['B'] = B
    _params['mu'] = mu
    
    ham = syst.hamiltonian_submatrix(params=_params, sparse=True)
    d = kwant.operator.Density(syst)
    e, ev = spectrum.sparse_diag(ham,4,0)
    return np.sort(np.abs(e))[::2]

In [ ]:
learner = adaptive.Learner2D(vary_mu_B, [[0, 100], [0, 4]])

In [ ]:
runner = adaptive.Runner(learner, executor=client)
runner.live_info()

In [ ]:
hv.notebook_extension('bokeh')

In [ ]:
import formulas as f
mu = np.linspace(0, 100)
vf = f.fermi_velocity(mu, 0.02)
eth = .5*f.thouless_energy(vf, 500)/params['g_factor_middle']/params['mu_B']

In [ ]:
%%opts Image [colorbar=True logz=False aspect=1] 
%%opts Curve (color='red')

# %%output filename='muB_gap_larger_range'
hv.HoloMap(learner.plot().redim(x='mu', y='B')*hv.Curve((mu, eth)), kdims='nth lowest energy')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
ax=plt.axes()
plotting_results.plot_syst(syst=syst, params=dict(**params),ax=ax)
kwant.plotter.map(syst, d(ev[:,0]), ax=ax)
# plt.savefig('no_ohase_difference', dpi=1000)

In [ ]:
from itertools import product
discrete_variations = dict()
discrete_variations['z_x'] = [700, 800, 900] #[500, 600, 700, 800, 900, 1000, 1100, 1200]
discrete_variations['z_y'] = [0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
discrete_variations['B'] = np.linspace(.2, .6, 21)

pars = {}

for z_x, z_y, B in product(*discrete_variations.values()):
    _syst_pars = syst_pars.copy()
    _params = params.copy()
    _syst_pars['z_x'] = z_x
    _syst_pars['L_x'] = (10000//z_x)*z_x
    _syst_pars['z_y'] = z_y
    _params['B'] = B
    pars[(z_x, z_y, B)] = (_syst_pars, _params)

In [ ]:
res = lview.map(spectrum.calc_lowest_state, pars.values())

In [ ]:
res.done()




 
# res.result()



In [ ]:
len(res.submitted)

In [ ]:
res.progress


In [ ]:
res.result()

In [ ]:
data=dict(zip(pars.keys(), res.result()))

In [ ]:
N_z_x = len(discrete_variations['z_x'])
N_z_y = len(discrete_variations['z_y'])
N_B = len(discrete_variations['B'])
Z_0s = {}
Z_1s = {}

for B in discrete_variations['B']:
    Z_0 = np.zeros((N_z_x, N_z_y))
    Z_1 = np.zeros((N_z_x, N_z_y))
    for idx_x, z_x in enumerate(discrete_variations['z_x']):
        for idx_y, z_y in enumerate(discrete_variations['z_y']):
            key = (z_x, z_y, B)
            Z_0[idx_x, idx_y] = data[key][0]
            Z_1[idx_x, idx_y] = data[key][1]
            if z_x == 1000 and z_y == 125:
                print(B, data[key][0], data[key][1])
    Z_0s[B] = Z_0
    Z_1s[B] = Z_1
    

In [ ]:
m = 0
for k,v in data.items():
    if max(v) > m and k[1]==300:
        print(k, v)
        m=max(v)

In [ ]:
plot_dict = {('lowest', k): hv.Image(np.flip(v,1).T, bounds=[discrete_variations['z_x'][0], discrete_variations['z_y'][0],
                          discrete_variations['z_x'][-1],discrete_variations['z_y'][-1]]) for k,v in Z_0s.items()}
plot_dict.update({('second lowest', k): hv.Image(np.flip(v,1).T, bounds=[discrete_variations['z_x'][0], discrete_variations['z_y'][0],
                          discrete_variations['z_x'][-1],discrete_variations['z_y'][-1]]) for k,v in Z_1s.items()})

In [ ]:
hv.notebook_extension('matplotlib')

In [ ]:
z_x = [k[0] for k in data if k[2]==discrete_variations['B'][5]]
z_y = [k[1] for k in data if k[2]==discrete_variations['B'][5]]
I = [v[1] for k,v in data.items() if k[2]==discrete_variations['B'][5]]

In [ ]:
z_x

In [ ]:
i=scipy.interpolate.interp2d(z_x, z_y, I)

In [ ]:
i(800, 0)

In [ ]:
hv.HoloMap({'a':plt.pcolor(Z_0)})

In [ ]:
%%opts Image [colorbar=True width=400 logz=False] (cmap='Viridis') {+axiswise}
# %%output filename='500nm_70percent_transparancy'
(hv.HoloMap(plot_dict, kdims=['which eigenmode', 'B'])).redim(x='z_x', y='z_y')#.redim.range(z=(0,0.1))

In [ ]:
Z_

In [ ]:
%opts?

In [ ]:
print(res.done(), res.started, res.elapsed)


In [ ]:
def bands_wrap(k):
    ham = syst.hamiltonian_submatrix(sparse=True, params=dict(**params, k_x=k))
    e, ev = spectrum.sparse_diag(ham, 100, 0)
    return np.sort(e)
    
learner = adaptive.Learner1D(bands_wrap, [-np.pi, np.pi])
runner = adaptive.Runner(learner)
runner.live_info()

In [ ]:
learner.plot()

In [ ]:
def lowest_e_at_B(B):
    _params = params.copy()
    _params['B'] = B
    ham = syst.hamiltonian_submatrix(params=_params, sparse=True)
    d = kwant.operator.Density(syst)
    e, ev = spectrum.sparse_diag(ham,4,0)
    return np.sort(np.abs(e))[::2]

In [ ]:
learner = adaptive.Learner1D(lowest_e_at_B, [0.2, .4])

In [ ]:
runner = adaptive.Runner(learner, ntasks=15)
runner.live_info()

In [ ]:
learner.plot()
